In [5]:
import gzip
from collections import defaultdict

def readGz(f):
  for l in gzip.open(f):
    yield eval(l)

In [2]:
### Would-visit baseline: just rank which businesses are popular and which are not, and return '1' if a business is among the top-ranked

businessCount = defaultdict(int)
totalPurchases = 0

for l in readGz("train.json.gz"):
  user,business = l['userID'],l['businessID']
  businessCount[business] += 1
  totalPurchases += 1

mostPopular = [(businessCount[x], x) for x in businessCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
  count += ic
  return1.add(i)
  if count > totalPurchases/2: break

predictions = open("predictions_Visit.txt", 'w')
for l in open("pairs_Visit.txt"):
  if l.startswith("userID"):
    #header
    predictions.write(l)
    continue
  u,i = l.strip().split('-')
  if i in return1:
    predictions.write(u + '-' + i + ",1\n")
  else:
    predictions.write(u + '-' + i + ",0\n")

predictions.close()

NameError: name 'defaultdict' is not defined

In [1]:
reviews = []
for l in readGz("train.json.gz"):
    reviews.append(l)

NameError: name 'readGz' is not defined

In [144]:
userList = []
businessList = []
#user->business hashtable
userHash = {}
#business->user hashtable
businessHash = {}
#list of pairs
pairs = []

for v in reviews:
    userID = v['userID']
    businessID = v['businessID']
    pairs.append((userID,businessID))
    if businessID not in businessHash:
        businessHash[businessID] = {}
        businessList.append(businessID)
    if userID not in userHash:
        userHash[userID] = {}
        userList.append(userID)
    if userID not in businessHash[businessID]:
        businessHash[businessID][userID] = 0
    if businessID not in userHash[userID]:
        userHash[userID][businessID] = 0
        
        
    userHash[userID][businessID] += 1
    businessHash[businessID][userID] += 1

import random
f_pairs = []
while len(f_pairs) != 200000:
    user = random.choice(userList)
    business = random.choice(businessList)
    if user not in businessHash[business]:
        f_pairs.append((user,business))
print userHash["U093387342"]
train_true_pairs = pairs[:100000]
train_false_pairs = f_pairs[:100000]
valid_true_pairs = pairs[100000:]
valid_false_pairs = f_pairs[100000:]

{'B211117692': 1, 'B744259798': 1, 'B161779289': 1, 'B652388772': 1, 'B071971841': 1, 'B775853642': 1}


In [128]:
### #1 ###
businessCount = defaultdict(int)
totalPurchases = 0

#Modified baseline for training set
for p in train_true_pairs:
    user,business = p[0],p[1]
    businessCount[business] += 1
    totalPurchases += 1

mostPopular = [(businessCount[x], x) for x in businessCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
predictor = {}
for ic, i in mostPopular:
    count += ic
    predictor[i] = True
    if count > totalPurchases/2: break

In [129]:
TP,TN,FP,FN = 0,0,0,0
for v in valid_true_pairs:
    if(v[1] in predictor):
        TP += 1
    else:
        FN += 1
    
for v in valid_false_pairs:
    if(v[1] in predictor):
        FP += 1
    else:
        TN += 1
        
print TP, TN, FP, FN
accuracy = (TP + TN) / float(TP + TN + FP + FN)
print accuracy

44216 80828 19172 55784
0.62522


In [130]:
#### Find better threshold

In [131]:
### #3 ###
#user->business hashtable
pred_user = {}
#business->user hashtable
pred_business = {}

for v in reviews[:100000]:
    userID = v['userID']
    businessID = v['businessID']
    # if user/business aren't in table add them
    if businessID not in pred_business:
        pred_business[businessID] = {}
    if userID not in pred_user:
        pred_user[userID] = {}
        
    for category in v['categories']:
        # for all categories in the review, add categories to user & business
        if category not in pred_user[userID]:
            pred_user[userID][category] = True
        if category not in pred_business[businessID]:
            pred_business[businessID][category] = True

def prediction(pair):
    userID, businessID = pair
    pred = False
    #if both user & business have been seen before
    if userID in pred_user and businessID in pred_business:
        # for each category for the user, check if the business matches. 
        for cat in pred_user[userID]:
            if cat in pred_business[businessID]:
                pred = True
    return pred

In [132]:
TP,TN,FP,FN = 0,0,0,0
for v in valid_true_pairs:
    if(prediction(v)):
        TP += 1
    else:
        FN += 1

    
for v in valid_false_pairs:
    if(prediction(v)):
        FP += 1
    else:
        TN += 1
    
print TP, TN, FP, FN
accuracy = (TP + TN) / float(TP + TN + FP + FN)
print accuracy

49121 80860 19140 50879
0.649905


In [133]:
predictions = open("predictions_Visit.txt", 'w')
for l in open("pairs_Visit.txt"):
    if l.startswith("userID"):
        #header
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    if prediction((u,i)):
        predictions.write(u + '-' + i + ",1\n")
    else:
        predictions.write(u + '-' + i + ",0\n")

predictions.close()

In [201]:
### Category prediction baseline: Just consider some of the most common words from each category

catDict = {
  "American Restaurant": 0,
  "Bar": 1,
  "Asian Restaurant": 2,
  "European Restaurant": 3,
  "Italian Restaurant": 4,
  "Fast Food Restaurant": 5,
  "Mexican Restaurant": 6,
  "Seafood Restaurant": 7,
  "Coffee Shop": 8,
  "Sandwich Shop": 9
}
catList = ["American Restaurant",
  "Bar",
  "Asian Restaurant",
  "European Restaurant",
  "Italian Restaurant",
  "Fast Food Restaurant",
  "Mexican Restaurant",
  "Seafood Restaurant",
  "Coffee Shop",
  "Sandwich Shop"]

predictions = open("predictions_Category.txt", 'w')
predictions.write("userID-reviewHash,category\n")
for l in readGz("test_Category.json.gz"):
  cat = catDict['American Restaurant'] # If there's no evidence, just choose the most common category in the dataset
  words = l['reviewText'].lower()
  if 'america' in words:
    cat = catDict['American Restaurant']
  if 'bar' in words or 'beer' in words:
    cat = catDict['Bar']
  if 'asia' in words:
    cat = catDict['Asian Restaurant']
  if 'europe' in words:
    cat = catDict['European Restaurant']
  if 'italian' in words:
    cat = catDict['Italian Restaurant']
  if 'fast' in words:
    cat = catDict['Fast Food Restaurant']
  if 'mexic' in words:
    cat = catDict['Mexican Restaurant']
  if 'coffee' in words:
    cat = catDict['Coffee Shop']
  if 'sandwich' in words:
    cat = catDict['Sandwich Shop']
  predictions.write(l['userID'] + '-' + l['reviewHash'] + "," + str(cat) + "\n")

predictions.close()

In [162]:
### #5 ###
import numpy as np

reviews = []
for l in readGz("train.json.gz"):
    l["category"] = np.array([0] * 10)
    for k,v in catDict.iteritems():
        if k in l["categories"]:
            l["category"][v] = 1
    if sum(l["category"]) == 1:
        reviews.append(l)
            
train = reviews[: len(reviews)/2]       
valid = reviews[len(reviews)/2 :]

70195


In [166]:
userCats = {}
for r in train:
    userID = r['userID']
    if userID not in userCats:
        userCats[userID] = np.array([0] * 10)
    userCats[userID] += r["category"]
def predictor(r,users):
    userID = r["userID"]
    if userID in users:
        return np.argmax(users[userID])
    else:
        return 0

In [167]:
T,F = 0,0
for v in valid:
    if(predictor(v,userCats) == v['categoryID'] ):
        T += 1
    else:
        F += 1
    
print T,F
print T / float(T + F)

10380 24718
0.295743347199


In [188]:
from nltk.tokenize import TweetTokenizer
tokenizer = TweetTokenizer()
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
cat_words = [{} for i in range(10)]
tot_words = {}
for r in train:
    cat = r['categoryID']
    revWords = tokenizer.tokenize(r['reviewText'])
    revWords = [stemmer.stem(w) for w in revWords]
    for word in revWords:
        if word not in tot_words:
            tot_words[word] = 0
        tot_words[word] += 1
        if word not in cat_words[cat]:
            cat_words[cat][word] = 0
        cat_words[cat][word] += 1

In [189]:
import operator
it = iter(sorted(tot_words.items(),key=operator.itemgetter(1),reverse=True))
mostcommon = []
for i in range(500):
    mostcommon.append(it.next())
    
print mostcommon

{
    "saimin": 3, 
    "miyazaki": 1, 
    "mid-week": 2, 
    "yellow": 32, 
    "four": 40, 
    "harrowdown": 1, 
    "prefix": 1, 
    "roslyn": 1, 
    "over-r": 2, 
    "mantwo": 1, 
    "lord": 2, 
    "wizardri": 1, 
    "uofc": 1, 
    "deli": 3, 
    "furikak": 2, 
    "foul": 2, 
    "sweetbread": 1, 
    "heilig-levin": 1, 
    "fus": 1, 
    "elvi": 4, 
    "prize": 1, 
    "wooden": 9, 
    "showcas": 5, 
    "wednesday": 7, 
    "suhi": 1, 
    "super-fresh": 1, 
    "fritter": 11, 
    "tumer": 1, 
    "karedok": 1, 
    "parchment-pap": 1, 
    "citrus-dil": 1, 
    "\u7c73\u56fd\u3067\u3042\u308b\u3053\u3068\u3092\u8003\u3048\u308b\u3068": 1, 
    "bacon": 18, 
    "unfriendli": 1, 
    "budget": 9, 
    "mind-boggl": 1, 
    "second": 67, 
    "sooth": 4, 
    "maloney": 1, 
    "sushiopoli": 1, 
    "gujarati": 1, 
    "springrol": 1, 
    "dahi": 2, 
    "specialist": 1, 
    "hero": 2, 
    "swad": 1, 
    "herb": 4, 
    "error": 1, 
    "mentaiko": 1, 
    "mas

In [196]:
probs = [{} for i in range(10)]
for cat in range(10):
    for word, tot in mostcommon:
        if word in cat_words[cat]:
            probs[cat][word] = cat_words[cat][word] / float(tot)
        else:
            probs[cat][word] = 0
print probs

[{u'all': 0.30239374694675136, u':)': 0.27163461538461536, u'ice': 0.29174311926605506, u'go': 0.30431814302782045, u'to': 0.3064492961792588, u'yummi': 0.28365384615384615, u'late': 0.3231597845601436, u'leav': 0.36674816625916873, u'worth': 0.348762109795479, u'me': 0.2821505376344086, u'everi': 0.3510167992926614, u'far': 0.27167630057803466, u'dessert': 0.48376259798432253, u'fan': 0.2676056338028169, u'veri': 0.30139152435167615, u'ramen': 0.009765625, u'cool': 0.2650056625141563, u'tri': 0.29502237136465326, u'salad': 0.4195501730103806, u'did': 0.37032085561497324, u'list': 0.4204724409448819, u'item': 0.36724137931034484, u'small': 0.26531986531986534, u'enjoy': 0.31305903398926654, u'pork': 0.25193370165745854, u'neighborhood': 0.2805232558139535, u'tea': 0.10377358490566038, u'bacon': 0.6173708920187794, u'chef': 0.3952941176470588, u'dish': 0.277260018639329, u'street': 0.20642201834862386, u'even': 0.3161290322580645, u'what': 0.3066173448417591, u'chicago': 0.3430962343096

In [204]:
it = [iter(sorted(probs[cat].items(),key=operator.itemgetter(1),reverse=True)) for cat in range(10)]

for cat in range(10):
    print catList[cat], "|","Probability"
    print "---|---"
    for i in range(10):
        w = it[cat].next()
        print w[0],"|", w[1]
    print

American Restaurant | Probability
---|---
diner | 0.791666666667
pancak | 0.734553775744
brunch | 0.660714285714
breakfast | 0.657849829352
bacon | 0.617370892019
egg | 0.525161290323
cake | 0.515274949084
potato | 0.490774907749
french | 0.490683229814
burger | 0.488877904103

Bar | Probability
---|---
bartend | 0.518771331058
game | 0.492937853107
play | 0.488977955912
music | 0.478887744593
show | 0.442238267148
bar | 0.435890410959
beer | 0.38759936407
hang | 0.359447004608
drink | 0.357699070585
cool | 0.345413363533

Asian Restaurant | Probability
---|---
ramen | 0.984375
indian | 0.968864468864
thai | 0.961357210179
chines | 0.95777027027
noodl | 0.94170984456
curri | 0.93347639485
dumpl | 0.918629550321
sushi | 0.896857373086
rice | 0.725274725275
spici | 0.664719626168

European Restaurant | Probability
---|---
pizza | 0.313004484305
chicago | 0.135983263598
potato | 0.108856088561
wine | 0.105882352941
bread | 0.104022191401
chees | 0.0966149506347
french | 0.0931677018634
be

In [206]:
print reviews[0]

{'category': array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'rating': 4.0, 'reviewHash': 'R567271252', 'businessID': 'B423621081', 'unixReviewTime': 1378865039, 'reviewText': u"I'm a vegetarian, but every so often I want a hotdog with lots of toppings.  And a tall can of beer.  Frank has got that covered.  And they have a cool warehouse space with some pinball machines.  Prices are a little high for hotdogs...fancy hotdogs, but hotdogs nonetheless.  Good location and service, but gets crowded and loud.", 'userID': 'U985379327', 'reviewTime': u'Sep 10, 2013', 'categories': [u'American Restaurant', u'Cafe', u'Hot Dog Restaurant'], 'categoryID': 0}
